<div class="contentcontainer med left" style="margin-left: -50px;">
<dl class="dl-horizontal">
  <dt>Title</dt> <dd> RadialHeatMap Element</dd>
  <dt>Dependencies</dt> <dd>Bokeh</dd>
  <dt>Backends</dt> <dd><a href='./RadialHeatMap.ipynb'>Bokeh</a></dd> <dd><a href='../matplotlib/RadialHeatMap.ipynb'>Matplotlib</a></dd>
</dl>
</div>

In [ ]:
import pandas as pd
import holoviews as hv
hv.extension('bokeh')

``RadialHeatMap`` is well suited to discover **periodic patterns** and **trends** in **time series** data. 

### NYC Taxi Data

Let's dive into a concrete example, namely the New York - Taxi Data ([For-Hire Vehicle (“FHV”) records](http://www.nyc.gov/html/tlc/html/about/trip_record_data.shtml)). The following data contains hourly pickup counts for the entire year of 2016. 

**Considerations**: Thinking about taxi pickup counts, we might expect higher taxi usage during business hours. In addition, public holidays should be clearly distinguishable from regular business days. Furthermore, we might expect high taxi pickup counts during Friday and Saterday nights.

**Design**: In order model the above ideas, we decide to assign days with hourly split to the *radial segments* and week of year to the *annulars*. This will allow to detect daily/hourly periodicity and weekly trends. To get you more familiar with the mapping of segemnts and annulars, take a look at the following radial heatmap:

In [ ]:
%%opts RadialHeatMap [width=900 height=900 tools=["hover"]]

# load example data
df_nyc = pd.read_csv("../../../assets/nyc_taxi_2016.csv", parse_dates=["Pickup_date"])

# create relevant time columns
df_nyc["Day & Hour"] = df_nyc["Pickup_date"].dt.strftime("%A %H:00")
df_nyc["Week of Year"] = df_nyc["Pickup_date"].dt.strftime("Week %W")
df_nyc["Date"] = df_nyc["Pickup_date"].dt.strftime("%Y-%m-%d")

hv.RadialHeatMap(df_nyc, kdims=["Day & Hour", "Week of Year"], vdims=["Pickup_Count", "Date"])

**At first glance**: First, let's take a closer look at the mentioned segments and annulars. **Segments** correspond to *hours of a given day* whereas **annulars** represent entire *weeks*. If you use the hover tool, you will quickly get an idea of how segments and annulars are organized. **Color** decodes the pickup values with blue being low and red being high.

**Plot improvements**: The above plot clearly shows systematic patterns however the default plot options are somewhat disadvantageous. Therefore, before we start to dive into the results, let's increase the readability of the given plot:

- **Remove annular ticks**: The information about week of year is not very important. Therefore, we hide it via `yticks=None`.
- **Custom segment ticks**: Right now, segment labels are given via day and hour. We don't need hourly information and we want every day to be labeled. We can use a tuple here which will be passed to `xticks=("Friday", ..., "Thursday")`
- **Add segment markers**: Moreover, we want to aid the viewer in distingushing each day more clearly. Hence, we can provide marker lines via `xmarks=7`.
- **Rotate heatmap**: The week starts with Monday and ends with Sunday. Accordingly, we want to rotate the plot to have Sunday and Monday be at the top. This can be done via `start_angle=np.pi*19/14`. The default order is defined by the global sort order which is present in the data. The default starting angle is at 12 o'clock.

Let's see the result of these modifications:

In [ ]:
%%opts RadialHeatMap [width=900 height=900 tools=["hover"] yticks=None xmarks=7 start_angle=np.pi*19/14]
%%opts RadialHeatMap [xticks=("Friday", "Saterday", "Sunday", "Monday", "Tuesday", "Wednesday", "Thursday")]

hv.RadialHeatMap(df_nyc, kdims=["Day & Hour", "Week of Year"], vdims=["Pickup_Count", "Date"])

After tweaking the plot defaults, we're comfortable with the given visualization and can focus on the story the plot tells us.

**There are many interesting findings in this visualization:**

1. Taxi pickup counts are high between 7-9am and 5-10pm during weekdays which business hours as expected. In contrast, during weekends, there is not much going on until 11am. 
2. Friday and Saterday nights clearly stand out with the highest pickup densities as expected. 
3. Public holidays can be easily identified. For example, taxi pickup counts are comparetively low around Christmas and Thanksgiving.
4. Weather phenomena also influence taxi service. There is a very dark blue stripe at the beginning of the year starting at Saterday 23rd and lasting until Sunday 24th. Interestingly, there was one of the [biggest blizzards](https://www.weather.gov/okx/Blizzard_Jan2016) in the history of NYC.

### Global surface temperatures

Lets take a look at a different example. We want visualize global surface temperatures ([source](http://berkeleyearth.org/data/)) in degree Celsius using a radial heatmap:

In [ ]:
def mark_annulars(x):
    if int(x) % 100 == 0:
        return True

In [ ]:
%%opts RadialHeatMap [width=900 height=900 tools=["hover"] xticks=12 yticks=mark_annulars xmarks=4 ymarks=mark_annulars]
%%opts RadialHeatMap (ymarks_line_width=2 xmarks_line_width=2)

df = pd.read_csv("../../../assets/temperatures.csv")
hv.RadialHeatMap(df, kdims=["month", "year"], vdims="temp_absolute")

Each **segment** corresponds to a *month* whereas each **annular** represents a *year*. We passed a custom function to `ymarks` and `yticks` to define which annulars we to mark and label. In addition, we set `xticks=12` in order to show all month segment labels.

The periodic pattern of winter months yielding lower temperatures than sommer months becomes clearly evident. Interestingly, one has to take a closer look to discover the effect of global warming of the last decades. The intermediate months April and October indicate the trend of rising temperatures via the change from blue to red color. In order to make this trend even more vivid, one can normalize the absolute monthly temperatures as anomalies relative to the Jan 1951-Dec 1980 average (see more [here](https://bobtisdale.wordpress.com/2014/11/09/on-the-elusive-absolute-global-mean-surface-temperature-a-model-data-comparison/)) and choose a different time range beginning with 1900:

In [ ]:
%%opts RadialHeatMap [width=900 height=900 tools=["hover"] xticks=12]

hv.RadialHeatMap(df.query("year > 1900"), kdims=["month", "year"], vdims="temp_anomaly")